## TASK 1

In [26]:
import pandas as pd
import numpy as np

In [20]:
#merge the dataframes

url_a_test = 'Data/arguments-test.tsv'
df_a_test = pd.read_csv(url_a_test, sep='\t')

url_a_training = 'Data/arguments-training.tsv'
df_a_training = pd.read_csv(url_a_training, sep='\t')

url_a_validation = 'Data/arguments-validation.tsv'
df_a_validation = pd.read_csv(url_a_validation, sep='\t')

url_l_test = 'Data/labels-test.tsv'
df_l_test = pd.read_csv(url_l_test, sep='\t')

url_l_training = 'Data/labels-training.tsv'
df_l_training = pd.read_csv(url_l_training, sep='\t')

url_l_validation = 'Data/labels-validation.tsv'
df_l_validation = pd.read_csv(url_l_validation, sep='\t')


In [21]:
#merge argument dataframes with label dataframes
df_test = pd.merge(df_a_test, df_l_test, on='Argument ID')

df_training = pd.merge(df_a_training, df_l_training, on='Argument ID')

df_validation = pd.merge(df_a_validation, df_l_validation, on='Argument ID')

In [ ]:
#merge level 2 annotations to level 3 categories

df_test['Openess to change'] = df_test['Self-direction: thought'] | df_test['Self-direction: action'] | df_test['Stimulation'] | df_test['Hedonism'] 
df_test['Self-enhancement'] = df_test['Hedonism'] | df_test['Achievement'] | df_test['Power: dominance'] | df_test['Power: resources'] | df_test['Face']
df_test['Conservation'] = df_test['Face'] | df_test['Security: personal'] | df_test['Security: societal'] | df_test['Tradition'] | df_test['Conformity: rules'] | df_test['Conformity: interpersonal'] | df_test['Humility']
df_test['Self-transcendence'] = df_test['Humility'] | df_test['Benevolence: caring'] | df_test['Benevolence: dependability'] | df_test['Universalism: concern'] | df_test['Universalism: nature']| df_test['Universalism: tolerance'] | df_test['Universalism: objectivity']

df_training['Openess to change'] = df_training['Self-direction: thought'] | df_training['Self-direction: action'] | df_training['Stimulation'] | df_training['Hedonism']
df_training['Self-enhancement'] = df_training['Hedonism'] | df_training['Achievement'] | df_training['Power: dominance'] | df_training['Power: resources'] | df_training['Face']
df_training['Conservation'] = df_training['Face'] | df_training['Security: personal'] | df_training['Security: societal'] | df_training['Tradition'] | df_training['Conformity: rules'] | df_training['Conformity: interpersonal'] | df_training['Humility']
df_training['Self-transcendence'] = df_training['Humility'] | df_training['Benevolence: caring'] | df_training['Benevolence: dependability'] | df_training['Universalism: concern'] | df_training['Universalism: nature']| df_training['Universalism: tolerance'] | df_training['Universalism: objectivity']

df_validation['Openess to change'] = df_validation['Self-direction: thought'] | df_validation['Self-direction: action'] | df_validation['Stimulation'] | df_validation['Hedonism']
df_validation['Self-enhancement'] = df_validation['Hedonism'] | df_validation['Achievement'] | df_validation['Power: dominance'] | df_validation['Power: resources'] | df_validation['Face']
df_validation['Conservation'] = df_validation['Face'] | df_validation['Security: personal'] | df_validation['Security: societal'] | df_validation['Tradition'] | df_validation['Conformity: rules'] | df_validation['Conformity: interpersonal'] | df_validation['Humility']
df_validation['Self-transcendence'] = df_validation['Humility'] | df_validation['Benevolence: caring'] | df_validation['Benevolence: dependability'] | df_validation['Universalism: concern'] | df_validation['Universalism: nature']| df_validation['Universalism: tolerance'] | df_validation['Universalism: objectivity']

df_validation.head()

In [ ]:
#drop unuseful columns
df_test = df_test.drop(columns=['Self-direction: thought', 'Self-direction: action', 'Stimulation', 'Hedonism', 'Achievement', 'Power: dominance', 'Power: resources', 'Face', 'Security: personal', 'Security: societal', 'Tradition', 'Conformity: rules', 'Conformity: interpersonal', 'Humility', 'Benevolence: caring', 'Benevolence: dependability', 'Universalism: concern', 'Universalism: nature', 'Universalism: tolerance', 'Universalism: objectivity'])
df_training = df_training.drop(columns=['Self-direction: thought', 'Self-direction: action', 'Stimulation', 'Hedonism', 'Achievement', 'Power: dominance', 'Power: resources', 'Face', 'Security: personal', 'Security: societal', 'Tradition', 'Conformity: rules', 'Conformity: interpersonal', 'Humility', 'Benevolence: caring', 'Benevolence: dependability', 'Universalism: concern', 'Universalism: nature', 'Universalism: tolerance', 'Universalism: objectivity'])
df_validation = df_validation.drop(columns=['Self-direction: thought', 'Self-direction: action', 'Stimulation', 'Hedonism', 'Achievement', 'Power: dominance', 'Power: resources', 'Face', 'Security: personal', 'Security: societal', 'Tradition', 'Conformity: rules', 'Conformity: interpersonal', 'Humility', 'Benevolence: caring', 'Benevolence: dependability', 'Universalism: concern', 'Universalism: nature', 'Universalism: tolerance', 'Universalism: objectivity'])

In [25]:
df_validation.head()

,Argument ID,Conclusion,Stance,Premise,Openess to change,Self-enhancement,Conservation,Self-transcendence
0,A01001,Entrapment should be legalized,in favor of,if entrapment can serve to more easily capture...,0,0,1,0
1,A01012,The use of public defenders should be mandatory,in favor of,the use of public defenders should be mandator...,0,0,0,1
2,A02001,Payday loans should be banned,in favor of,payday loans create a more impoverished societ...,0,0,1,1
3,A02002,Surrogacy should be banned,against,Surrogacy should not be banned as it is the wo...,1,0,0,0
4,A02009,Entrapment should be legalized,against,entrapment is gravely immoral and against huma...,0,0,1,1


## TASK 2 - MODEL DEFINITION

In [38]:
#categories 
categories = ['Openess to change', 'Self-enhancement', 'Conservation', 'Self-transcendence']

In [39]:
def create_random_binary_classifier(category):
    """
    Crea un classificatore binario casuale che predice solo la categoria specificata.

    :param category: La categoria che il classificatore predirà
    :return: Una funzione che genera previsioni casuali della categoria specificata
    """
    def random_binary_classifier(size):
        """
        Genera previsioni casuali della categoria specificata.

        :param size: Numero di previsioni da generare
        :return: Array di previsioni casuali della categoria specificata
        """
        return np.random.choice([0, 1], size=size)
    
    return random_binary_classifier

In [40]:
for category in categories:
    globals()[f'random_binary_classifier_{category}'] = create_random_binary_classifier(category)


In [41]:
for category in categories:
    classifier = globals()[f'random_binary_classifier_{category}']
    df_test[f'predicted_{category}'] = classifier(len(df_test))

In [43]:
df_test.head()

,Argument ID,Conclusion,Stance,Premise,Openess to change,Self-enhancement,Conservation,Self-transcendence,predicted_Openess to change,predicted_Self-enhancement,predicted_Conservation,predicted_Self-transcendence
0,A26004,We should end affirmative action,against,affirmative action helps with employment equity.,0,1,1,1,1,1,0,0
1,A26010,We should end affirmative action,in favor of,affirmative action can be considered discrimin...,0,1,0,1,1,1,0,0
2,A26016,We should ban naturopathy,in favor of,naturopathy is very dangerous for the most vul...,0,1,1,1,0,0,1,0
3,A26024,We should prohibit women in combat,in favor of,women shouldn't be in combat because they aren...,0,1,0,0,1,1,1,1
4,A26026,We should ban naturopathy,in favor of,once eradicated illnesses are returning due to...,0,1,1,1,1,0,0,0


In [49]:
print(globals()[f'random_binary_classifier_Openess to change'])

<function create_random_binary_classifier.<locals>.random_binary_classifier at 0x1398013a0>
